In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

test1.csv  train.csv


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

fraud_prediction7_6(1).ipynb  fraud_prediction7_6.ipynb
fraud_prediction7_6(2).ipynb


In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 16.7MB/s eta 0:00:01


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
'''
    找到关键特征才是王道
    构造多个新特征，特征比大于10，Score=???
    Score=?
'''
import pandas as pd
train=pd.read_csv('./data/train.csv')
test1=pd.read_csv('./data/test1.csv')
train

In [ ]:
features=train.drop(['Unnamed: 0','label'],axis=1)
labels=train['label']
features.columns

In [ ]:
train.info()
# 查看缺失值
train.isnull().sum()

In [ ]:
# 数据探索，找到导致1的关键特征值
def find_key_feature(train,selected):
    temp0=train[train['label']==0]
    temp=pd.DataFrame(columns=[0,1])
    temp[0]=temp0[selected].value_counts()/len(temp0)*100
    temp1=train[train['label']==1]
    temp[1]=temp1[selected].value_counts()/len(temp1)*100
    temp[2]=temp[1]/temp[0]
    # 选出大于10倍的特征
    result=temp[temp[2]>10].sort_values(2,ascending=False).index
    return result
key_feature={}
key_feature['osv']=find_key_feature(train,'osv')
key_feature

In [ ]:
features.columns

In [ ]:
selected_cols=['osv', 'apptype', 'carrier', 'dev_height', 'dev_ppi','dev_width', 'lan', 'media_id', 'ntt', 'os', 'package', 'sid','timestamp', 'version', 'fea_hash', 'location', 'fea1_hash','cus_type']
for selected in selected_cols:
    key_feature[selected]=find_key_feature(train,selected)
key_feature

In [ ]:
# 构造新特征，新特征字段=原始特征字段+1
def f(x,selected):
    # 判断是否在关键特征值里，是1，否0
    if x in key_feature[selected]:
        return 1
    else:
        return 0

for selected in selected_cols:
    if len(key_feature[selected])>0:
        features[selected+'1']=features[selected].apply(f,args=(selected, ))
        test1[selected+'1']=test1[selected].apply(f,args=(selected, ))
        print(selected+'1 created!!!')

In [ ]:
features['osv1'].value_counts()

In [ ]:
features[['osv1','apptype1','dev_height1','dev_ppi1','dev_width1','media_id1','ntt1','package1','fea_hash1','fea1_hash1']]

In [ ]:
# 查看每个字段唯一值的个数，唯一值个数很少，说明是类别特征
for feature in  features.columns:
    print(feature,features[feature].nunique())
# 确定类别特征
cate_features=['apptype','carrier','ntt','version','location','cus_type']

In [ ]:
import time # Python底层的日期函数
from datetime import datetime # 高级封装接口

def get_date(features):
    # 除以1000,转换为日期格式
    features['timestamp']=features['timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
    # 创建时间戳索引
    temp=pd.DatetimeIndex(features['timestamp'])
    features['year']=temp.year
    features['month']=temp.month
    features['day']=temp.day
    features['week_day']=temp.weekday
    features['hour']=temp.hour
    features['minute']=temp.minute
    
    # 添加time_diff
    start_time=features['timestamp'].min()
    features['time_diff']=features['timestamp']-start_time   
    # 将time_diff转换为小时
    features['time_diff']=features['time_diff'].dt.days*24+features['time_diff'].dt.seconds/3600
    # Thinking：是否需要这么多特征？？？
    # 使用day,time_diff
    features.drop(['timestamp','year','month','week_day','minute'],axis=1,inplace=True)
    return features

# 对训练集提取时间多尺度
features=get_date(features)
# 对测试集提取时间多尺度
test1=get_date(test1)

In [ ]:
# 对OSV进行LabelEncoder
# 还需要转换为Float，需要做数据清洗
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
# 需要将训练集和测试集合并，然后统一做LabelEncoder
all_df=pd.concat([train,test1])
all_df['osv']=all_df['osv'].astype('str')
all_df['osv']=le.fit_transform(all_df['osv'])
all_df['osv']

In [ ]:
# 不参与建模的特征['os','lan','sid']
# os为Android,android 实际上是唯一的，去掉
remove_list=['os','lan','sid']
col=features.columns.tolist()
for i in remove_list:
    col.remove(i)
col

In [ ]:
# carries属于类别特征
features['carrier'].value_counts()

In [ ]:
# 特征筛选
features=features[col]
features

In [ ]:
# 特征变换,对于数值过大的异常值 设置为0
features['fea_hash']=features['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
features['fea_hash']

In [ ]:
features['fea1_hash']=features['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
features['fea1_hash']

In [ ]:
features['version'].value_counts()

In [ ]:
# 对数据清洗，将V3=>3,V1=>1,V6=>6,V2=>2
# 针对version,非数值类型 设置0
features['version']=features['version'].map(lambda x:int(x) if str(x).isdigit() else 0)
features['version']

In [ ]:
features['osv']=all_df[all_df['label'].notnull()]['osv']
features['osv']

In [ ]:
# 类别类型，需要数值
features[cate_features]

In [ ]:
features

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score

def ensemble_model(clf,train_x,train_y,test,cate_features):
    # 采用五折交叉验证 ensemble model
    sk=StratifiedKFold(n_splits=5,shuffle=True,random_state=2021)
    prob=[]   # 记录最终结果
    mean_acc=0   #记录平均准确率
    for k,(train_index,val_index) in enumerate(sk.split(train_x,train_y)):
        train_x_real=train_x.iloc[train_index]
        train_y_real=train_y.iloc[train_index]
        val_x=train_x.iloc[val_index]
        val_y=train_y.iloc[val_index]
        # 子模型训练
        clf=clf.fit(train_x_real,train_y_real,categorical_feature=cate_features)
        val_y_pred=clf.predict(val_x)
        acc_val=accuracy_score(val_y,val_y_pred)
        # 子模型评估
        print('第{}个子模型acc:{}'.format(k+1,acc_val))
        mean_acc+=acc_val/5
        # 子模型预测0，1
        test_y_pred=clf.predict_proba(test)[:,-1]  # soft 得到概率值
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob=sum(prob)/5
    return mean_prob

In [ ]:
# 测试集预测，保持与features中的columns一致
test_fea=test1[features.columns]

# 特征变换,对于数值过大的异常值 设置为0
test_fea['fea_hash']=test_fea['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
test_fea['fea1_hash']=test_fea['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
# 对数据清洗，将V3=>3,V1=>1,V6=>6,V2=>2
# 针对version,非数值类型 设置0
test_fea['version']=test_fea['version'].map(lambda x:int(x) if str(x).isdigit() else 0)
test_fea['osv']=all_df[all_df['label'].isnull()]['osv']

In [ ]:
import lightgbm as lgb
# 使用LightGBM训练
# 相比XGBoost的改进：对类别特征的处理，不需要进行one-hot
clf=lgb.LGBMClassifier(
    num_leaves=2**5-1, 
    reg_alpha=0.25, 
    reg_lambda=0.25, 
    objective='binary',
    max_depth=-1, 
    learning_rate=0.005, 
    min_child_samples=3, 
    random_state=2021,
    n_estimators=10000, 
    subsample=1, 
    colsample_bytree=1
)
result=ensemble_model(clf,features,labels,test_fea,cate_features)
result

In [ ]:
# 保存结果
a=pd.DataFrame(test1['sid'])
a['label']=result
# 转换为二分类
a['label']=a['label'].apply(lambda x:0 if x<0.5 else 1)
a.to_csv('./data/baseline.csv',index=False)